In [ ]:
from core.RR import seg_gpt,rule_compose_stage_1,rule_compose_stage_2,rule_compose_stage_3
import os

In [ ]:
if not os.listdir("data/tmp"):
    os.makedirs("data/tmp")
todo_list = ["train","dev","test"]
for todo_name in tqdm(todo_list):
    with open("data/origin_data/CHIP-CDN_"+todo_name+".json","r",encoding="utf-8") as f:
        data_list = json.load(f)
    # if todo_name == "test":
    #     start=3999
    # else:
    #     start=1
    result_list=[]
    for item in tqdm(data_list):
        tmp=seg_gpt(item["text"],item["normalized_result"])
        with open("data/tmp/"+todo_name+".jsonl","a",encoding="utf-8") as f:
            f.write(json.dumps(tmp,ensure_ascii=False))
            f.write("\n")
        time.sleep(10)
        # time.sleep(30)

In [ ]:
for todo_name in todo_list:
    data_list=[]
    with open("data/tmp/"+todo_name+".jsonl","r",encoding="utf-8") as f:
        line = f.readline().strip()
        data_list.append(json.loads(line))
    with open("data/1_recognization_result/"+todo_name+".json","w",encoding="utf-8") as f:
        f.write(json.dumps(data_list,ensure_ascii=False,indent=2))

In [ ]:
for todo_name in todo_list:
    data_list = json.load(open("data/1_recognization_result/"+todo_name,"r",encoding="utf-8"))
    result_list = []
    for v in data_list:
        result_list.append({
            "text":v["text"],
            "normalized_result":v["normalized_result"],
            "RR":rule_compose_stage_1(v["text"],v["label_result"])
        })
    with open("data/tmp/"+todo_name+".json","w",encoding="utf-8") as f:
        f.write(json.dumps(result_list,indent=2,ensure_ascii=False))

In [ ]:
for todo_name in todo_list:
    data_list = json.load(open("data/tmp/"+todo_name+".json","r",encoding="utf-8"))
    result_list = []
    for v in data_list:
        result_list.append({
            "text":v["text"],
            "normalized_result":v["normalized_result"],
            "RR":rule_compose_stage_2(v["text"],v["RR"])
        })
    with open("data/tmp/"+todo_name+".json","w",encoding="utf-8") as f:
        f.write(json.dumps(result_list,indent=2,ensure_ascii=False))

In [ ]:
for todo_name in todo_list:
    data_list = json.load(open("data/tmp/"+todo_name+".json","r",encoding="utf-8"))
    result_list = []
    for v in data_list:
        s3_res = rule_compose_stage_3(v["text"],v["RR"])
        result_list.append({
            "text":v["text"],
            "normalized_result":v["normalized_result"],
            "RR":s3_res if len(s3_res)>0 else v["RR"]
        })
    with open("data/2_restructure_result/"+todo_name+".json","w",encoding="utf-8") as f:
        f.write(json.dumps(result_list,indent=2,ensure_ascii=False))